# Prelude

In [1]:
import sys
from operator import add
import json
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF, CountVectorizer, ChiSqSelector, StringIndexer, Normalizer
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
import numpy as np
import re
import math
from pathlib import Path
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType, BooleanType, ArrayType
from pyspark.sql.functions import rand, split, posexplode, explode, col, collect_list, struct
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import pandas as pd

In [2]:
# Global constants
data_dir = 'recsys19/'

In [3]:
# Initialize the spark session used for all calculations
spark = SparkSession \
    .builder \
    .appName('recsys19') \
    .getOrCreate()

# Data loading


In [4]:
df_meta = spark.read.csv(
    data_dir + 'df_metadata.csv',
    header=True,
)

schema_meta = StructType() \
    .add('pandas_id', IntegerType()) \
    .add('item_id', StringType())


for subtype in df_meta.schema:
    name = subtype.name
    
    if name in ('_c0', 'item_id'):
        continue
        
    schema_meta = schema_meta.add(
        StructField(name, IntegerType()))

df_meta = spark.read.csv(
    data_dir + 'df_metadata.csv',
    header=True,
    schema=schema_meta
)

df_meta = df_meta.drop('pandas_id')

df_meta.first()

Row(item_id='5101', 1 Star=0, 2 Star=0, 3 Star=0, 4 Star=1, 5 Star=0, Accessible Hotel=1, Accessible Parking=1, Adults Only=0, Air Conditioning=1, Airport Hotel=0, Airport Shuttle=1, All Inclusive (Upon Inquiry)=0, Balcony=1, Bathtub=1, Beach=0, Beach Bar=0, Beauty Salon=0, Bed & Breakfast=0, Bike Rental=1, Boat Rental=0, Body Treatments=0, Boutique Hotel=0, Bowling=1, Bungalows=0, Business Centre=1, Business Hotel=1, Cable TV=1, Camping Site=0, Car Park=1, Casa Rural (ES)=0, Casino (Hotel)=0, Central Heating=1, Childcare=0, Club Hotel=0, Computer with Internet=1, Concierge=0, Conference Rooms=1, Convenience Store=0, Convention Hotel=0, Cosmetic Mirror=1, Cot=1, Country Hotel=0, Deck Chairs=0, Design Hotel=0, Desk=1, Direct beach access=0, Diving=0, Doctor On-Site=0, Eco-Friendly hotel=0, Electric Kettle=1, Excellent Rating=0, Express Check-In / Check-Out=1, Family Friendly=1, Fan=1, Farmstay=0, Fitness=1, Flatscreen TV=1, Free WiFi (Combined)=0, Free WiFi (Public Areas)=0, Free WiFi (

In [5]:
schema_sessions = StructType([
    StructField("pandas_id", IntegerType()),
    StructField("user_id", StringType()),
    StructField("session_id", StringType()),
    StructField("timestamp", StringType()),
    StructField("step", IntegerType()),
    StructField("action_type", StringType()),
    StructField("reference", StringType()),
    StructField("platform", StringType()),
    StructField("city", StringType()),
    StructField("device", StringType()),
    StructField("current_filters", StringType()),
    StructField("impressions", StringType()),
    StructField("prices", StringType()),
    StructField("is_validation", BooleanType()),
    StructField("is_train", BooleanType()),
])

df_sessions = spark.read.csv(
    data_dir + 'df_sessions_small.csv',
    header=True,
    schema=schema_sessions
)

df_sessions = df_sessions.drop('pandas_id')

df_sessions.first()


Row(user_id='004A07DM0IDW', session_id='1d688ec168932', timestamp='2018-11-07 01:53:34+00:00', step=1, action_type='interaction item image', reference='2059240', platform='CO', city='Santa Marta, Colombia', device='mobile', current_filters=None, impressions=None, prices=None, is_validation=False, is_train=False)

# Preprocessing

## Calculate the facets

In [6]:
def preprocess_clickout(df):
    df = df.selectExpr('session_id', 'reference AS item_id')
    
    df = df.join(
        df_meta,
        'item_id'
    )
    
    df = df.drop('item_id')
    
    return df

    
facets = {
    # '': ('interaction item image',),
    'clickout': ('clickout item', ),
    # '': ('search for destination',),
    # 'search_item': ('search for item',),
    # '': ('interaction item rating',),
    # '': ('search for poi',),
    # '': ('interaction item info',),
    # '': ('interaction item deals',),
    # '': ('change of sort order',),
    # 'filter': ('filter selection',),
}


facet_data = {}


for name, params in facets.items():
    print(f'Computing facet {name}')
    
    action_name, = params

    # Select the rows applicable to this facet
    facet_in = df_sessions.where(f"action_type == '{action_name}'")

    # Apply the facet's preprocessing function
    preproc_function = globals()[f'preprocess_{name}']
    facet_out = preproc_function(facet_in)

    facet_data[name] = facet_out
    # facet_out.show(2)

Computing facet clickout


## Merge the facets into the full dataset

In [7]:
# Non-facet columns taken straight from the input
print('Generating base dataframe')
base_column_names = {
    'user_id', 'session_id'
}

session_ids = df_sessions.filter('action_type = "clickout item"').select('session_id').distinct()

# session_ids = df_sessions.select([c for c in df_sessions.columns if c in base_column_names]).distinct().persist()

# # Columns supplied by the facets
# for name, subdata in facet_data.items():
#     print(f'Adding {name} facet')
    
#     # Add a prefix to the column names of the subdata
#     subdata = subdata.toDF(*((f'{name}_{c}' if c != 'session_id' else c) for c in subdata.columns ))
    
# #     subdata.show(5)
# #     break
    
#     # Join the dataframes
#     data = data.join(
#         subdata,
#         on='session_id',
#         how='left_outer'
#     )
    
#     break
    
session_ids.show(5)

Generating base dataframe
+-------------+
|   session_id|
+-------------+
|d73eac31de33a|
|e2934eabdfcd3|
|7ab99b292eb5e|
|800946183890b|
|ad23d70233546|
+-------------+
only showing top 5 rows



## Multiplex the impressions

In [8]:
all_cols      = df_sessions.filter('action_type = "clickout item"').cache()
data_cols     = all_cols.select('session_id', 'platform', 'city', 'device', 'is_validation', 'is_train', 'reference')
distinct_cols = all_cols.select('session_id', 'impressions').distinct()


# Add columns from the raw input. They were removed before so the 'distinct' operation doesn't operate on them
df_joined = distinct_cols.join(
    data_cols,
    'session_id'
)


# Per-impression processing

# Explode the impressions
df_joined = df_joined.withColumn(
    'impressions',
    split(col('impressions'), '\\|')
)

df_joined = df_joined.selectExpr(
    '*',
    'posexplode(impressions) as (impression_index, impression_id)',
).drop('impressions')


# Add the ground truth
df_joined = df_joined.withColumn(
    'gt',
    (df_joined.impression_id == df_joined.reference).cast('float')
).drop('reference')


# Add the per-impression metadata
prefixed_df_meta = df_meta.toDF(*((f'gt_{c}' if c != 'item_id' else c) for c in df_meta.columns ))


df_joined = df_joined.join(
    prefixed_df_meta,
    on=df_joined.impression_id == prefixed_df_meta.item_id
).drop('item_id')


# All columns that are used for training the ML model
feature_col_names = [cn for cn in df_joined.columns if cn.startswith('gt_')]
print('Feature columns: ', feature_col_names)

# Merge the ML features into a single vector
assembler = VectorAssembler(
    inputCols=feature_col_names,
    outputCol="ml_features")

df_joined = assembler.transform(df_joined)

# df_joined.show(5)
df_joined.cache()

df_joined.printSchema()

Feature columns:  ['gt_1 Star', 'gt_2 Star', 'gt_3 Star', 'gt_4 Star', 'gt_5 Star', 'gt_Accessible Hotel', 'gt_Accessible Parking', 'gt_Adults Only', 'gt_Air Conditioning', 'gt_Airport Hotel', 'gt_Airport Shuttle', 'gt_All Inclusive (Upon Inquiry)', 'gt_Balcony', 'gt_Bathtub', 'gt_Beach', 'gt_Beach Bar', 'gt_Beauty Salon', 'gt_Bed & Breakfast', 'gt_Bike Rental', 'gt_Boat Rental', 'gt_Body Treatments', 'gt_Boutique Hotel', 'gt_Bowling', 'gt_Bungalows', 'gt_Business Centre', 'gt_Business Hotel', 'gt_Cable TV', 'gt_Camping Site', 'gt_Car Park', 'gt_Casa Rural (ES)', 'gt_Casino (Hotel)', 'gt_Central Heating', 'gt_Childcare', 'gt_Club Hotel', 'gt_Computer with Internet', 'gt_Concierge', 'gt_Conference Rooms', 'gt_Convenience Store', 'gt_Convention Hotel', 'gt_Cosmetic Mirror', 'gt_Cot', 'gt_Country Hotel', 'gt_Deck Chairs', 'gt_Design Hotel', 'gt_Desk', 'gt_Direct beach access', 'gt_Diving', 'gt_Doctor On-Site', 'gt_Eco-Friendly hotel', 'gt_Electric Kettle', 'gt_Excellent Rating', 'gt_Expre

In [9]:
# Train the estimator
train_rows = df_joined.filter(df_joined.gt.isNotNull())

estimator = LinearRegression(featuresCol='ml_features', labelCol='gt', maxIter=10)
estimator_model = estimator.fit(train_rows)

print(estimator_model.intercept, estimator_model.coefficients[:5], '...')

0.02579847917189499 [-0.00304866  0.00295211  0.00128796 -0.00025276 -0.00305285] ...


In [10]:
# Generate the predictions DF
# Make predictions
df_preds = df_joined.filter(df_joined.gt.isNull())

df_preds = estimator_model.transform(df_preds)

# Gather the impression rows
df_preds = df_preds.withColumn(
    'pairs',
    struct(['impression_id', 'prediction'])
)
    
df_preds = df_preds.groupBy('session_id').agg(collect_list('pairs').alias('collected'))

# Postprocess with pandas
df_preds = df_preds.toPandas()

df_preds['item_recommendations'] = df_preds.collected.apply(lambda x: ' '.join(str(y[0]) for y in x))
df_preds['item_probs'] = df_preds.collected.apply(lambda x: ' '.join(str(y[1]) for y in x))

df_preds.drop(columns=['collected'], inplace=True)

df_preds.head()


,session_id,item_recommendations,item_probs
0,106d7775a1b0a,5880336 10256276 8381638 9975198 8017628 69935...,0.023272669429515915 0.021752516709684335 0.02...
1,62b62eca1eb81,1054956 6338722 5841682 1177976 3132756 267645...,0.03295892838763685 0.05111944548814107 0.0246...
2,66b97824f9fa2,2630950 962199 3249068 5977428 3133564 29829 8...,0.025401344892967134 0.020218234436721355 0.02...
3,ef831aabb03c7,8258 8249 8243 8257 8266 8255 8268 8270 8244 8...,0.04103465523331417 0.03434411542241153 0.0451...
4,35a1b1854d56e,2717947 3860198 2674986 6257852 2673221 774755...,0.027364719571551795 0.023609639843930025 0.02...


In [12]:
df_results = df_sessions.select(
    'user_id',
    'session_id',
    'timestamp',
    'step',
   ).filter(df_sessions.reference.isNull()) \
    .filter('action_type = "clickout item"') \
    .distinct() \
    .toPandas()
    
print(len(df_results))

df_results = df_results.merge(
    df_preds,
    on='session_id'
)

df_results['timestamp'] = pd.to_datetime(df_results.timestamp).astype(np.int64) // 10 ** 9

df_results.head()

12856


,user_id,session_id,timestamp,step,item_recommendations,item_probs
0,XL8TSOP3B1YX,25724f8a2d3cf,1541557929,3,1132609 9604828 2506890 9221346 1137629 106251...,0.029086740715807923 0.02084500509973619 0.028...
1,Q756UKSL40C1,dbb2af2d92c27,1541558110,2,3244680 4359818 2780833 127128 1963235 3135079...,0.0382070451700988 0.023509841054655915 0.0330...
2,ZM4GWY6U7FA9,a8d32b9a7cad9,1541558598,3,1752861 6008738 1753385 2751588 2492902 118973...,0.03799671644167142 0.02084500509973619 0.0268...
3,AKEEN1Z79HYD,8934f035c99ee,1541552821,47,2146110 2146110 1010937 8756218 3895194 222294...,0.015085701405349572 0.015085701405349572 0.02...
4,0E6L1CJN4OW0,d7ecfca8a5dfc,1541557844,1,1938045 4992894 219266 9028118 7943108 6698132...,0.029960417340562703 0.03527061447210804 0.028...


In [13]:
df_results.to_csv('session_based.csv')